# Importing libraries

In [ ]:
import os, glob, json, csv, subprocess, sys, re
from urllib.request import urlopen
from itertools import chain
import urllib.request
from pprint import pprint
from bs4 import BeautifulSoup
from git import *
from subprocess import Popen, PIPE
from os import path
from collections import Counter
import pandas as pd
import itertools

# Defining directories

In [ ]:
userhome = os.path.expanduser('~')
repository = userhome + r'/different-diff/dataset/openjpa/'
analyze_dir = userhome + r'/different-diff/analyze/analyze_openjpa/'

# Defining git command function

In [ ]:
def execute_shell_command(buglink, id1, id2, cmd, work_dir):
    #Executes a shell command in a subprocess, waiting until it has completed.
    pipe = subprocess.Popen(cmd, shell=True, cwd=work_dir, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
    (out, error) = pipe.communicate()
    return buglink, id1, id2, out, error
    pipe.wait()

# Loading file of commit id - parent id

In [ ]:
header = ['bug_id','bugintro_commitID','parent_id']
parent_id = pd.read_csv(analyze_dir + '02_extracting_commitid/parentid_of_bugintro.csv')
parent_id = parent_id[header]
parent_id

# Applying -w --ignore-blank-lines --diff-algorithm=histogram

In [ ]:
diffhist = []
for v in range(0, len(parent_id)):
    sys.stdout.write('\rExtracting parent id for commits: %i' %(v+1) + ' out of %i' %(len(parent_id)))
    sys.stdout.flush()
    w = parent_id.iloc[v]
    diff_cmd = "git diff -w --ignore-blank-lines --diff-algorithm=histogram " + w[2] + " " + w[1] + " --numstat"
    tmp = (str(execute_shell_command(w[0], w[1] , w[2], diff_cmd, repository)).replace("(",'').
               replace("+",'').replace("\\n",',').replace("(+)",'').replace("(-)",'').replace("\\t",',').
               replace(")",'').replace("|",'').replace(" ",""))
    tmp = re.sub("([a-z])" and "([ ])", r"\1,", tmp)
    tmp2 = (str(tmp).replace(",b'",',').replace(", ",'').replace("\'",'').replace(",,",''))
    diffhist.append([str(tmp2)])
    
diffhistresult = []
for item in diffhist:
    if item == ', ':
        del item
    else:
        diffhistresult.append(item)
        
print ("\nExtraction is complete")

In [ ]:
klhist = []
reshist = []
for v in range(0,len(diffhistresult)):
    for w in range(0,len(diffhistresult[v])):
        klhist = diffhistresult[v][w].split(',')
        reshist.append(klhist)

In [ ]:
with open(analyze_dir + '03_file-diff/diffhist-bugs.csv', 'w') as csvfile:
    header = ['bug_id','bugintro_commitID', 'parent_id', '#insertions','#deletions','filename','#line_changed']
    writers = csv.writer(csvfile, delimiter=',')
    writers.writerow(header)
    for m in range(0,len(reshist)):
        i = 3
        j = 4
        k = 5
        for x in range(0,int((len(reshist[m])-3)/3)):
            if (reshist[m][i]) == "-":
                pass
            else:
                line_changed = int(reshist[m][i]) + int(reshist[m][j])
                try:
                    pattern = re.search(('({\w.*=>|{=>)'), reshist[m][k])
                    pattern = pattern.groups()[0]
                    v = [reshist[m][0], reshist[m][1], reshist[m][2], reshist[m][i], reshist[m][j], reshist[m][k].replace(pattern,"").replace("}",""), str(line_changed)]
                except:             
                    v = [reshist[m][0], reshist[m][1], reshist[m][2], reshist[m][i], reshist[m][j], reshist[m][k], str(line_changed)]
                writers.writerow(v)
            i = i + 3
            j = j + 3
            k = k + 3
            
print ("The file 'diffhist-bugs.csv' has been created")

# Applying -w --ignore-blank-lines --diff-algorithm=minimal

In [ ]:
diffmin = []
for v in range(0, len(parent_id)):
    sys.stdout.write('\rExtracting parent id for commits: %i' %(v+1) + ' out of %i' %(len(parent_id)))
    sys.stdout.flush()
    w = parent_id.iloc[v]
    diff_cmd = "git diff -w --ignore-blank-lines --diff-algorithm=minimal " + w[2] + " " + w[1] + " --numstat"
    tmp = (str(execute_shell_command(w[0], w[1] , w[2], diff_cmd, repository)).replace("(",'').
               replace("+",'').replace("\\n",',').replace("(+)",'').replace("(-)",'').replace("\\t",',').
               replace(")",'').replace("|",'').replace(" ",""))
    tmp = re.sub("([a-z])" and "([ ])", r"\1,", tmp)
    tmp2 = (str(tmp).replace(",b'",',').replace(", ",'').replace("\'",'').replace(",,",''))
    diffmin.append([str(tmp2)])
    
diffminresult = []
for item in diffmin:
    if item == ', ':
        del item
    else:
        diffminresult.append(item)
        
print ("\nExtraction is complete")

In [ ]:
klmin = []
resmin = []
for v in range(0,len(diffminresult)):
    for w in range(0,len(diffminresult[v])):
        klmin = diffminresult[v][w].split(',')
        resmin.append(klmin)

In [ ]:
with open(analyze_dir + '03_file-diff/diffmin-bugs.csv', 'w') as csvfile:
    header = ['bug_id','bugintro_commitID', 'parent_id', '#insertions','#deletions','filename','#line_changed']
    writers = csv.writer(csvfile, delimiter=',')
    writers.writerow(header)
    for m in range(0,len(resmin)):
        i = 3
        j = 4
        k = 5
        for x in range(0,int((len(resmin[m])-3)/3)):
            if (resmin[m][i]) == "-":
                pass
            else:
                line_changed = int(resmin[m][i]) + int(resmin[m][j])
                try:
                    pattern = re.search(('({\w.*=>|{=>)'), resmin[m][k])
                    pattern = pattern.groups()[0]
                    v = [resmin[m][0], resmin[m][1], resmin[m][2], resmin[m][i], resmin[m][j], resmin[m][k].replace(pattern,"").replace("}",""), str(line_changed)]
                except:             
                    v = [resmin[m][0], resmin[m][1], resmin[m][2], resmin[m][i], resmin[m][j], resmin[m][k], str(line_changed)]
                writers.writerow(v)
            i = i + 3
            j = j + 3
            k = k + 3
            
print ("The file 'diffmin-bugs.csv' has been created")

# Applying -w --ignore-blank-lines --diff-algorithm=myers

In [ ]:
diffmyers = []
for v in range(0, len(parent_id)):
    sys.stdout.write('\rExtracting parent id for commits: %i' %(v+1) + ' out of %i' %(len(parent_id)))
    sys.stdout.flush()
    w = parent_id.iloc[v]
    diff_cmd = "git diff -w --ignore-blank-lines --diff-algorithm=myers " + w[2] + " " + w[1] + " --numstat"
    tmp = (str(execute_shell_command(w[0], w[1] , w[2], diff_cmd, repository)).replace("(",'').
               replace("+",'').replace("\\n",',').replace("(+)",'').replace("(-)",'').replace("\\t",',').
               replace(")",'').replace("|",'').replace(" ",""))
    tmp = re.sub("([a-z])" and "([ ])", r"\1,", tmp)
    tmp2 = (str(tmp).replace(",b'",',').replace(", ",'').replace("\'",'').replace(",,",''))
    diffmyers.append([str(tmp2)])
    
diffmyersresult = []
for item in diffmyers:
    if item == ', ':
        del item
    else:
        diffmyersresult.append(item)
        
print ("\nExtraction is complete")

In [ ]:
klmyers = []
resmyers = []
for v in range(0,len(diffmyersresult)):
    for w in range(0,len(diffmyersresult[v])):
        klmyers = diffmyersresult[v][w].split(',')
        resmyers.append(klmyers)

In [ ]:
with open(analyze_dir + '03_file-diff/diffmyers-bugs.csv', 'w') as csvfile:
    header = ['bug_id','bugintro_commitID', 'parent_id', '#insertions','#deletions','filename','#line_changed']
    writers = csv.writer(csvfile, delimiter=',')
    writers.writerow(header)
    for m in range(0,len(resmyers)):
        i = 3
        j = 4
        k = 5
        for x in range(0,int((len(resmyers[m])-3)/3)):
            if (resmyers[m][i]) == "-":
                pass
            else:
                line_changed = int(resmyers[m][i]) + int(resmyers[m][j])
                try:
                    pattern = re.search(('({\w.*=>|{=>)'), resmyers[m][k])
                    pattern = pattern.groups()[0]
                    v = [resmyers[m][0], resmyers[m][1], resmyers[m][2], resmyers[m][i], resmyers[m][j], resmyers[m][k].replace(pattern,"").replace("}",""), str(line_changed)]
                except:             
                    v = [resmyers[m][0], resmyers[m][1], resmyers[m][2], resmyers[m][i], resmyers[m][j], resmyers[m][k], str(line_changed)]
                writers.writerow(v)
            i = i + 3
            j = j + 3
            k = k + 3
            
print ("The file 'diffmyers-bugs.csv' has been created")

# Applying -w --ignore-blank-lines --diff-algorithm=patience

In [ ]:
diffpat = []
for v in range(0, len(parent_id)):
    sys.stdout.write('\rExtracting parent id for commits: %i' %(v+1) + ' out of %i' %(len(parent_id)))
    sys.stdout.flush()
    w = parent_id.iloc[v]
    diff_cmd = "git diff -w --ignore-blank-lines --diff-algorithm=patience " + w[2] + " " + w[1] + " --numstat"
    tmp = (str(execute_shell_command(w[0], w[1] , w[2], diff_cmd, repository)).replace("(",'').
               replace("+",'').replace("\\n",',').replace("(+)",'').replace("(-)",'').replace("\\t",',').
               replace(")",'').replace("|",'').replace(" ",""))
    tmp = re.sub("([a-z])" and "([ ])", r"\1,", tmp)
    tmp2 = (str(tmp).replace(",b'",',').replace(", ",'').replace("\'",'').replace(",,",''))
    diffpat.append([str(tmp2)])
    
diffpatresult = []
for item in diffpat:
    if item == ', ':
        del item
    else:
        diffpatresult.append(item)
        
print ("\nExtraction is complete")

In [ ]:
klpat = []
respat = []
for v in range(0,len(diffpatresult)):
    for w in range(0,len(diffpatresult[v])):
        klpat = diffpatresult[v][w].split(',')
        respat.append(klpat)

In [ ]:
with open(analyze_dir + '03_file-diff/diffpat-bugs.csv', 'w') as csvfile:
    header = ['bug_id','bugintro_commitID', 'parent_id', '#insertions','#deletions','filename','#line_changed']
    writers = csv.writer(csvfile, delimiter=',')
    writers.writerow(header)
    for m in range(0,len(respat)):
        i = 3
        j = 4
        k = 5
        for x in range(0,int((len(respat[m])-3)/3)):
            if (respat[m][i]) == "-":
                pass
            else:
                line_changed = int(respat[m][i]) + int(respat[m][j])
                try:
                    pattern = re.search(('({\w.*=>|{=>)'), respat[m][k])
                    pattern = pattern.groups()[0]
                    v = [respat[m][0], respat[m][1], respat[m][2], respat[m][i], respat[m][j], respat[m][k].replace(pattern,"").replace("}",""), str(line_changed)]
                except:             
                    v = [respat[m][0], respat[m][1], respat[m][2], respat[m][i], respat[m][j], respat[m][k], str(line_changed)]
                writers.writerow(v)
            i = i + 3
            j = j + 3
            k = k + 3
            
print ("The file 'diffpat-bugs.csv' has been created")